In [1]:
import nltk as n
import pandas as pd
import nltk.tokenize
# n.download()

In [2]:
data = pd.read_csv("data.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, encoding = "latin-1", engine = "python")


In [3]:
data.describe()
data.head()


,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [4]:
line = data["STATUS"]
#line


#[n.word_tokenize(t) for t in n.sent_tokenize(line)]


Now, let's split words and sentences of our main column.

In [31]:
list_of_rows = []
words = []
sentences = []

list_of_rows = [l.split("\n")[0] for l in line ]


for k in list_of_rows:
    words.append(n.word_tokenize(k))
    sentences.append(n.sent_tokenize(k))
    
print("there are", len(words), "tokens and there are ", len(sentences))

print(sentences[:6])

#set(w.lower() for w in list_of_rows)


there are 9917 tokens and there are  9917
[['likes the sound of thunder.'], ["is so sleepy it's not even funny that's she can't get to sleep."], ['is sore and wants the knot of muscles at the base of her neck to stop hurting.', "On the other hand, YAY I'M IN ILLINOIS!", '<3'], ['likes how the day sounds in this new song.'], ['is home.', '<3'], ['www.thejokerblogs.com']]


In [6]:
fdist1 = n.FreqDist(list_of_rows)
print(fdist1)

fdist1.most_common(5)

<FreqDist with 9879 samples and 9917 outcomes>


[('black', 4),
 ('*PROPNAME*', 4),
 ('Merry Christmas everyone!', 3),
 ('Happy New Year!', 3),
 ('No one should die because they cannot afford health care, and no one should go broke because they get sick. If you agree, please post this as your status for the rest of the day.',
  3)]

In [8]:
# fdist1.hapaxes()
list = [l.split("\n")[0] for l in line ]

#tokens = n.word_tokenize(list)
#tokens

#snet = n.sent_tokenize(list_of_rows)
#len(snet)

In [9]:
list_of_rows[1]

"is so sleepy it's not even funny that's she can't get to sleep."

In [10]:
line.head()
print(line[1])

tag_words = n.pos_tag(words[1])
tag_words



#sent_words = n.pos_tag_sents(sentences)
#sent_words

is so sleepy it's not even funny that's she can't get to sleep.


[('is', 'VBZ'),
 ('so', 'RB'),
 ('sleepy', 'JJ'),
 ('it', 'PRP'),
 ("'s", 'VBZ'),
 ('not', 'RB'),
 ('even', 'RB'),
 ('funny', 'VBP'),
 ('that', 'DT'),
 ("'s", 'VBZ'),
 ('she', 'PRP'),
 ('ca', 'MD'),
 ("n't", 'RB'),
 ('get', 'VB'),
 ('to', 'TO'),
 ('sleep', 'VB'),
 ('.', '.')]

In [24]:
import sklearn as sk
train, test = sk.cross_validation.train_test_split(data, train_size = 0.8)


def word_fea(words):
    return dict((word, True) for word in words)

#Unigram features
word_fea(words[1])


{"'s": True,
 '.': True,
 'ca': True,
 'even': True,
 'funny': True,
 'get': True,
 'is': True,
 'it': True,
 "n't": True,
 'not': True,
 'she': True,
 'sleep': True,
 'sleepy': True,
 'so': True,
 'that': True,
 'to': True}

In [28]:
# Bigram collocation
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures as BAM
from itertools import chain
from nltk.book import text4

#text4.collocations("words")

def bigram_features(words, score_fn=BAM.chi_sq): 
    bg_finder = BigramCollocationFinder.from_words(words) 
    bigrams = bg_finder.nbest(score_fn, 100000) 
    return dict((bg, True) for bg in chain(words, bigrams)) 


bigram_features(words[1], score_fn=BAM.chi_sq)

TypeError: slice indices must be integers or None or have an __index__ method